Usando o Kmeans criado para visualizar os 5 clusters


In [4]:
import pandas as pd
import joblib

kmeans_model = joblib.load(r'C:\Users\apoll\OneDrive\Documentos\repositories\github\ssp_project\kmeans\kmeans_3.pkl')
scaler = joblib.load(r'C:\Users\apoll\OneDrive\Documentos\repositories\github\ssp_project\kmeans\scaler_3.pkl')

df = pd.read_csv(r"C:\Users\apoll\OneDrive\Documentos\repositories\github\ssp_project\data\processed\dados_alunos_mesclados_pivot.csv")

colunas_para_converter = ['vulnerabilidade_classe', "ameaca_classe"]
df = pd.get_dummies(df, columns=colunas_para_converter, drop_first=True)

for col in df.columns:
    if any(c in col for c in ['vulnerabilidade_classe', 'ameaca_classe']):
        df[col] = df[col].astype(int)

df = df.fillna(0)

numeric_df = df.select_dtypes(include=['number'])

numeric_df = numeric_df.reindex(columns=scaler.feature_names_in_, fill_value=0)

df_scaled = scaler.transform(numeric_df)

df['cluster'] = kmeans_model.predict(df_scaled)

cluster_col='cluster'
numeric_cols = numeric_df.columns.tolist()
cluster_profiles = df.groupby(cluster_col)[numeric_cols].mean()
mean_global = numeric_df.mean()
for c in sorted(df[cluster_col].unique()):
    diff = (cluster_profiles.loc[c] - mean_global[numeric_cols]).abs().sort_values(ascending=False)

    print(f"\n Top 5 variáveis que diferenciam o cluster {c}: ")
    print(diff.head(5))

    perguntas_destaque=diff.head(5).index.tolist()
    print(f"Perguntas mais relevantes para o cluster {c}: {perguntas_destaque}")
print("Quantidade de alunos por cluster: ")
print(df["cluster"].value_counts())


 Top 5 variáveis que diferenciam o cluster 0: 
Sabe informar se a família recebe algum benefício de transferência de renda do governo    0.390237
Sabe informar se a Renda familiar é menor do que 3 SM?                                    0.356607
O (a) estudante se identifica como preto(a) ou pardo(a)?                                  0.327380
Criança, adolescente ou jovem?                                                            0.282994
ameaca_classe_1. PEQUENA                                                                  0.259987
dtype: float64
Perguntas mais relevantes para o cluster 0: ['Sabe informar se a família recebe algum benefício de transferência de renda do governo', 'Sabe informar se a Renda familiar é menor do que 3 SM?', 'O (a) estudante se identifica como preto(a) ou pardo(a)?', 'Criança, adolescente ou jovem?', 'ameaca_classe_1. PEQUENA']

 Top 5 variáveis que diferenciam o cluster 1: 
vulnerabilidade_classe_1. BAIXA                                                

Visualizando o df com a coluna cluster 

In [5]:
df_cluster = pd.read_csv(r"C:\Users\apoll\OneDrive\Documentos\repositories\github\ssp_project\data\processed\dados_com_clusters.csv")


In [6]:
df_cluster.head()

,id,aluno_id,aluno_nome,turma,serie_ano_descricao,ensino_descricao,escola_descricao,si_descricao,avaliador,num_protocolo,...,vulnerabilidade_classe_2. MÉDIA,vulnerabilidade_classe_3. ALTA,vulnerabilidade_classe_4. MUITO ALTA,vulnerabilidade_classe_5. Pontuação fora da faixa.,ameaca_classe_1. PEQUENA,ameaca_classe_2. MODERADA,ameaca_classe_3. SIGNIFICANTE,ameaca_classe_4. EXTREMA,ameaca_classe_5. Pontuação fora da faixa.,cluster
0,114752,2022245041578,ENZO EMANUEL BRITO NASCIMENTO,222AI,2ª SÉRIE,MÉDIO REGULAR,CETI ZACARIAS DE GOIS,04ª GRE - TERESINA,CLAUDETE LOPES DA SILVA DE OLIVEIRA,AR2022245041578,...,0,0,0,0,0,0,0,0,0,0
1,114753,1719125490,FRANCISCA TAINARA RIBEIRO RODRIGUES DE SOUSA,222AI,2ª SÉRIE,MÉDIO REGULAR,CETI ZACARIAS DE GOIS,04ª GRE - TERESINA,CLAUDETE LOPES DA SILVA DE OLIVEIRA,AR1719125490,...,0,0,0,0,1,0,0,0,0,0
2,114754,2022245042280,JEOVANA CAROLINE DE SOUSA FEITOSA,222AI,2ª SÉRIE,MÉDIO REGULAR,CETI ZACARIAS DE GOIS,04ª GRE - TERESINA,CLAUDETE LOPES DA SILVA DE OLIVEIRA,AR2022245042280,...,0,0,0,0,1,0,0,0,0,0
3,114755,2022245042403,AMANDA VITORIA DA SILVA ANDRADE,222AI,2ª SÉRIE,MÉDIO REGULAR,CETI ZACARIAS DE GOIS,04ª GRE - TERESINA,CLAUDETE LOPES DA SILVA DE OLIVEIRA,AR2022245042403,...,0,0,0,0,1,0,0,0,0,0
4,114756,2022245048318,IZABEL FERNANDES SILVA,222AI,2ª SÉRIE,MÉDIO REGULAR,CETI ZACARIAS DE GOIS,04ª GRE - TERESINA,CLAUDETE LOPES DA SILVA DE OLIVEIRA,AR2022245048318,...,0,0,0,0,1,0,0,0,0,0


Gráficos


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

grouped = (
    df_cluster.groupby(['turma','escola_descricao','si_descricao','cluster']).size()
    .groupby(level=[0,1])
    .apply(lambda x: 100 * x / x.sum())
    .unspack(fill_value=0)
    )

turma_especifica = "118AI"
escola_especifica = "CETI AUGUSTINHO BRANDAO"

row = grouped.loc[(turma_especifica, escola_especifica)]

plt.figure(figsize=(6,4))
ax = row.plot(kind="bar")

plt.title(f"Turma: {turma_especifica} | Escola: {escola_especifica}")
plt.ylabel("Porcentagem de alunos (%)")
plt.xlabel("Cluster")
plt.ylim(0,100)
plt.xticks(rotation=0)
plt.tight_layout()

for p in ax.patches:
    ax.annotate(
        f"{p.get_height():.1f}%",
        (p)
    )




AttributeError: 'Series' object has no attribute 'unspack'

In [9]:
df_cluster[df_cluster['aluno_nome'] == 'GEOVANNA ALVES DA SILVA SOARES'] 

,id,aluno_id,aluno_nome,turma,serie_ano_descricao,ensino_descricao,escola_descricao,si_descricao,avaliador,num_protocolo,...,vulnerabilidade_classe_2. MÉDIA,vulnerabilidade_classe_3. ALTA,vulnerabilidade_classe_4. MUITO ALTA,vulnerabilidade_classe_5. Pontuação fora da faixa.,ameaca_classe_1. PEQUENA,ameaca_classe_2. MODERADA,ameaca_classe_3. SIGNIFICANTE,ameaca_classe_4. EXTREMA,ameaca_classe_5. Pontuação fora da faixa.,cluster
44155,174019,2022245131922,GEOVANNA ALVES DA SILVA SOARES,119AM,9º ANO,FUNDAMENTAL,CETI DOUTOR BARROSO,13ª GRE - SÃO RAIMUNDO NONATO,CLERISTON SANTANA VILANOVA,AR2022245131922,...,0,0,0,0,1,0,0,0,0,1
